In [1]:
import pandas as pd
import json
from geopy.distance import geodesic
import plotly.express as px
import plotly.graph_objects as go
import os

In [2]:
BASE_PATH = '../almrrc2021/almrrc2021-data-training/model_build_inputs/'

In [3]:
# DRONE CONSTANTS
DRONE_MAX_SPEED = 7  # m/s
DRONE_DEPTH = 30 # cm
DRONE_HEIGHT = 25 # cm
DRONE_WIDTH = 30 # cm
DRONE_LOADING = 15 # seconds
DRONE_LAUNCH = 5 # seconds
DRONE_LAND = 5 # seconds

In [4]:
# Loading all the package data

package_data = json.load(open(BASE_PATH + 'package_data.json'))

flattened_package_data = []
for route_id, stops in package_data.items():
    for stop_id, packages in stops.items():
        for package_id, details in packages.items():
            record = {
                "route_id": route_id,
                "stop_id": stop_id,
                "package_id": package_id,
                "scan_status": details["scan_status"] if "scan_status" in details else None, # note model_apply_inputs does not have scan_status, model_build_inputs does
                "start_time_utc": details["time_window"]["start_time_utc"],
                "end_time_utc": details["time_window"]["end_time_utc"],
                "planned_service_time_seconds": details["planned_service_time_seconds"],
                "depth_cm": details["dimensions"]["depth_cm"],
                "height_cm": details["dimensions"]["height_cm"],
                "width_cm": details["dimensions"]["width_cm"],
            }
            flattened_package_data.append(record)

package_data_df = pd.DataFrame(flattened_package_data)

package_data_df.head()

,route_id,stop_id,package_id,scan_status,start_time_utc,end_time_utc,planned_service_time_seconds,depth_cm,height_cm,width_cm
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AD,PackageID_9d7fdd03-f2cf-4c6f-9128-028258fc09ea,DELIVERED,NaN,NaN,59.3,25.4,7.6,17.8
1,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AD,PackageID_5541e679-b7bd-4992-b288-e862f6c84ae7,DELIVERED,2018-07-27 16:00:00,2018-07-28 00:00:00,59.3,25.4,12.7,17.8
2,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AD,PackageID_84d0295b-1adb-4a33-a65e-f7d6247c7a07,DELIVERED,NaN,NaN,59.3,39.4,7.6,31.8
3,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AF,PackageID_15c6a204-ec5f-4ced-9c3d-472316cc7759,DELIVERED,2018-07-27 16:00:00,2018-07-28 00:00:00,27.0,30.0,3.0,27.4
4,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AG,PackageID_3b28f781-242e-416e-9575-84c7188b8208,DELIVERED,NaN,NaN,45.0,25.4,12.7,17.8


In [5]:
# Loading all the route data

route_data = json.load(open(BASE_PATH + 'route_data.json'))

flattened_route_data = []

for route_id, info in route_data.items():
    for stop_id, stop_details in info['stops'].items():
        flattened_route_data.append({
            "route_id": route_id,
            "station_code": info['station_code'],
            "date": info['date_YYYY_MM_DD'],
            "departure_time_utc": info['departure_time_utc'],
            "executor_capacity_cm3": info['executor_capacity_cm3'],
            "route_score": info['route_score'],
            "stop_id": stop_id,
            "lat": stop_details['lat'],
            "lng": stop_details['lng'],
            "type": stop_details['type'],
            "zone_id": stop_details['zone_id']
        })

route_data_df = pd.DataFrame(flattened_route_data)

route_data_df.head()

,route_id,station_code,date,departure_time_utc,executor_capacity_cm3,route_score,stop_id,lat,lng,type,zone_id
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AD,34.099611,-118.283062,Dropoff,P-12.3C
1,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AF,34.101587,-118.291125,Dropoff,A-1.2D
2,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AG,34.089727,-118.285530,Dropoff,A-2.1A
3,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,BA,34.096132,-118.292869,Dropoff,A-1.2C
4,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,BE,34.098482,-118.286243,Dropoff,P-13.3B


In [6]:
vehicle_travel_times = json.load(open(BASE_PATH + 'travel_times.json'))

In [7]:
def get_data_for_route(route_id):
    package_data = package_data_df[package_data_df['route_id'] == route_id]
    route_data = route_data_df[route_data_df['route_id'] == route_id]
    travel_times_route = vehicle_travel_times[route_id]

    return package_data.copy(), route_data.copy(), travel_times_route.copy()

In [8]:
# Given route data, it will return a dictionary
def calculate_drone_travel_time(route_data):
  drone_travel_time = {}

  for _, base_row in route_data.iterrows():
    drone_travel_time[base_row['stop_id']] = {}

    # calculate travel time to every other stop in the route
    for _, other_row in route_data.iterrows():
      if base_row['stop_id'] != other_row['stop_id']:
        base_location = (base_row['lat'], base_row['lng'])
        other_location = (other_row['lat'], other_row['lng'])

        distance = geodesic(base_location, other_location).meters

        drone_travel_time[base_row['stop_id']][other_row['stop_id']] = distance / DRONE_MAX_SPEED
        # drone_travel_time[base_row['stop_id']][other_row['stop_id']] += 10 # 5 second rise, 5 second drop

      else:
        drone_travel_time[base_row['stop_id']][other_row['stop_id']] = 0

  return drone_travel_time

In [9]:
# Instead of getting a random route we will get a route for which we have the proposed sequence

with open('proposed_sequences.json') as file:
    proposed_sequences_json = json.load(file)

proposed_route_ids = list(proposed_sequences_json.keys())

In [10]:
route_id = proposed_route_ids[0]
package_data_route_df, route_data_route_df, vehicle_travel_times_route = get_data_for_route(route_id)
drone_travel_times_route = calculate_drone_travel_time(route_data_route_df)

In [11]:
# Determining if each package can be delivered by a drone
package_data_route_df['drone_possible'] = (package_data_route_df['depth_cm'] < DRONE_DEPTH) & (package_data_route_df['height_cm'] < DRONE_HEIGHT) & (package_data_route_df['width_cm'] < DRONE_WIDTH)

In [12]:
package_info_agg = package_data_route_df.groupby('stop_id').apply(
    lambda x: pd.Series({
        'packages': x[['package_id', 'depth_cm', 'height_cm', 'width_cm']].to_dict('records'),
        'num_packages': len(x),
        'drone_possible': (len(x) == 1) and (x['drone_possible'].iloc[0] == True),
        'service_time_seconds': x['planned_service_time_seconds'].mean() # Not sure if this should be sum or average. https://github.com/MIT-CAVE/rc-cli/blob/main/templates/data_structures.md
    })
).reset_index()

In [13]:
route_data_route_df = pd.merge(route_data_route_df, package_info_agg, how='left', on='stop_id')
route_data_route_df.head()

,route_id,station_code,date,departure_time_utc,executor_capacity_cm3,route_score,stop_id,lat,lng,type,zone_id,packages,num_packages,drone_possible,service_time_seconds
0,RouteID_a2c4695d-dcad-4975-84e6-c6b249c644ed,DCH2,2018-07-21,15:07:57,3313071.0,High,AK,41.998574,-87.801173,Dropoff,E-7.1A,[{'package_id': 'PackageID_45172842-41b8-411b-...,4.0,False,62.8
1,RouteID_a2c4695d-dcad-4975-84e6-c6b249c644ed,DCH2,2018-07-21,15:07:57,3313071.0,High,AL,41.996075,-87.792091,Dropoff,E-7.2B,[{'package_id': 'PackageID_0d19491c-6a2b-4dd3-...,1.0,False,77.0
2,RouteID_a2c4695d-dcad-4975-84e6-c6b249c644ed,DCH2,2018-07-21,15:07:57,3313071.0,High,AM,42.004423,-87.815098,Dropoff,E-6.1D,[{'package_id': 'PackageID_43c7fe5e-e0fd-4d15-...,3.0,False,15.0
3,RouteID_a2c4695d-dcad-4975-84e6-c6b249c644ed,DCH2,2018-07-21,15:07:57,3313071.0,High,AS,41.999975,-87.803708,Dropoff,E-7.1A,[{'package_id': 'PackageID_f97b5cbe-99fe-4cee-...,1.0,False,41.0
4,RouteID_a2c4695d-dcad-4975-84e6-c6b249c644ed,DCH2,2018-07-21,15:07:57,3313071.0,High,AT,41.998229,-87.795140,Dropoff,E-7.3B,[{'package_id': 'PackageID_7bce942f-9082-42af-...,1.0,False,50.0


In [14]:
proposed_sequences = json.load(open('proposed_sequences.json'))

In [15]:
def sequence_to_stops_for_route(route_id):

    route_sequence = proposed_sequences[route_id]['proposed']
    
    # Sorting them by their order
    route_sequence_sorted = dict(sorted(route_sequence.items(), key=lambda item: item[1]))

    # Converting keys to an array
    route_sequence_sorted = list(route_sequence_sorted.keys())

    # The last stop to the station (first stop) is not included in the sequence so we will manually add it
    route_sequence_sorted.append(route_sequence_sorted[0])

    return route_sequence_sorted

In [16]:
# Both
LOAD_START = 'LOAD_START'
LOAD_END = 'LOAD_END'
ARRIVE = 'ARRIVE'
DEPART = 'DEPART'

# Vehicle
SERVICE_START = 'SERVICE_START'
SERVICE_END = 'SERVICE_END'


# Drone
LAUNCH = 'LAUNCH'
DRONE_DELIVERY_START = 'DRONE_DELIVERY_START'
DRONE_DELIVERY_END = 'DRONE_DELIVERY_END'
LAND = 'LAND'

In [17]:
class Event:
  def __init__(self, event_type, stop_id, time, data): # data holds extra information, like coordinates
    self.event_type = event_type
    self.stop_id = stop_id
    self.time = time
    self.data = data

  def __str__(self):
    return f"{self.event_type} {self.stop_id} {round(self.time, 1)} {self.data}"

  def to_dict(self):
    # Convert the event to a dictionary
    return {
        'event_type': self.event_type,
        'stop_id': self.stop_id,
        'time': self.time,
        'coordinates': self.data,
    }

In [18]:
class VRPDSolver():
    def __init__(self, route_id, sequence, package_data, route_data, vehicle_travel_times, drone_travel_times):
        self.route_id = route_id
        self.sequence = sequence
        self.package_data = package_data
        self.route_data = route_data
        self.vehicle_travel_times = vehicle_travel_times
        self.drone_travel_times = drone_travel_times

        self.vehicle_events = []
        self.vehicle_path = []
        self.drone_events = []
        self.drone_path = []
        self.final_time = 0
    
    
    def find_drone_uses(self):
        drone_possible_sequence = []

        for stop_id in self.sequence:
            stop_row = self.route_data[self.route_data['stop_id'] == stop_id]
            
            drone_possible = 1 if stop_row['drone_possible'].values[0] else 0
            
            # can't use drone for station
            if stop_id == self.sequence[0]:
                drone_possible = 0
        
            drone_possible_sequence.append(drone_possible)

        return drone_possible_sequence
    
    def drone_save_time(self, drone_possible_sequence):
        time_saved = [-1]*len(self.sequence)
        for i in range(2, len(self.sequence) - 2):
            if drone_possible_sequence[i] == 1:
                vehicle_time = 0
                drone_time = 0
                vehicle_drone_time = 0
                
                vehicle_time += self.vehicle_travel_times[self.sequence[i-1]][self.sequence[i]]
                vehicle_time += self.vehicle_travel_times[self.sequence[i]][self.sequence[i+1]]
                vehicle_time += self.route_data[self.route_data['stop_id'] == self.sequence[i]]['service_time_seconds'].values[0]
                
                drone_time += DRONE_LOADING
                drone_time += DRONE_LAUNCH
                drone_time += self.drone_travel_times[self.sequence[i-1]][self.sequence[i]]
                drone_time += DRONE_LAND + DRONE_LAUNCH
                drone_time += self.drone_travel_times[self.sequence[i]][self.sequence[i+1]]
                drone_time += DRONE_LAND
                
                vehicle_drone_time += DRONE_LOADING
                vehicle_drone_time += self.vehicle_travel_times[self.sequence[i-1]][self.sequence[i+1]]
                
                max_time = max(vehicle_drone_time, drone_time)
                
                if max_time < vehicle_time:
                    time_saved[i] = round(vehicle_time - max_time, 2)
        
        return time_saved
    
    def maximize_saved_time(self, time_saved):
        n = len(time_saved)
        dp = [0] * (n + 1)
        for i in range(1, n + 1):
            dp[i] = max(time_saved[i - 1] + dp[i - 2], dp[i - 1])
        
        drone_usage = [0] * n
        i = n
        while i > 0:
            if dp[i] != dp[i - 1]:
                drone_usage[i - 1] = 1
                i -= 2
            else:
                i -= 1
        
        return drone_usage
    
    
    def generate_vehicle_drone_path(self, drone_usage):
        cur_coords = None
        cur_stop = None
        cur_time = 0
        
        drone_coords = None
        drone_time = 0
        
        cur_stop = self.sequence[0]
        stop = self.route_data[self.route_data['stop_id'] == cur_stop].iloc[0]
        cur_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}
        
        first_event = Event(DEPART, cur_stop, cur_time, cur_coords)
        self.vehicle_events.append(first_event)
        self.vehicle_path.append(cur_stop)
        self.drone_events.append(first_event)
        self.drone_path.append(cur_stop)
        

        
        i = 1
        while i < len(self.sequence):
            # Traveling to the next vehicle stop, departing stop based on if a drone was used
            if drone_usage[i - 1] == 1:
                cur_time += self.vehicle_travel_times[self.sequence[i - 2]][self.sequence[i]]
            else:
                cur_time += self.vehicle_travel_times[self.sequence[i - 1]][self.sequence[i]]
            
            # Arrive
            cur_stop = self.sequence[i]
            
            
            stop = self.route_data[self.route_data['stop_id'] == cur_stop].iloc[0]
            cur_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}
            arrive_event = Event(ARRIVE, cur_stop, cur_time, cur_coords)
            self.vehicle_events.append(arrive_event)
            self.vehicle_path.append(cur_stop)
            
            if cur_stop == self.sequence[-1]:
                self.drone_events.append(arrive_event)
                self.drone_path.append(cur_stop)
                break
         
            
            # The drone is flying and needs to land
            landed = True
            if drone_usage[i - 1] == 1:
                landed = False
            else:
                self.drone_events.append(arrive_event)
                self.drone_path.append(cur_stop)
            
            if not landed:
                # Checks if the vehicle is at the stop  
                if drone_time < cur_time:
                    landed = True
                    drone_land_event = Event(LAND, cur_stop, cur_time, cur_coords)
                    self.drone_events.append(drone_land_event)
                    self.drone_path.append(cur_stop)
            
            
            # Driver delivers package
            service_start_event = Event(SERVICE_START, cur_stop, cur_time, cur_coords)
            self.vehicle_events.append(service_start_event)
            cur_time += stop['service_time_seconds']
            service_end_event = Event(SERVICE_END, cur_stop, cur_time, cur_coords)
            self.vehicle_events.append(service_end_event)
            
            # If the drone didn't land, it will try again and make vehicle wait if needed
            if not landed:
                landed = True
                if drone_time < cur_time:
                    drone_land_event = Event(LAND, cur_stop, drone_time, cur_coords) # Note we can land on drone_time (when package is being delivered)
                    self.drone_events.append(drone_land_event)
                    self.drone_path.append(cur_stop)
                else:
                    cur_time = drone_time
                    drone_land_event = Event(LAND, cur_stop, cur_time, cur_coords)
                    self.drone_events.append(drone_land_event)
                    self.drone_path.append(cur_stop)
            
            
            
            # The next stop requires a drone
            if drone_usage[i + 1] == 1:
                
                # Driver loads the drone
                load_start_event = Event(LOAD_START, cur_stop, cur_time, cur_coords)
                self.vehicle_events.append(load_start_event)
                self.vehicle_events.append(load_start_event)
                cur_time += DRONE_LOADING
                load_end_event = Event(LOAD_END, cur_stop, cur_time, cur_coords)
                self.vehicle_events.append(load_end_event)
                self.drone_events.append(load_end_event)
                
                # Drone flies to the next stop
                drone_time = cur_time
                drone_launch_event = Event(LAUNCH, cur_stop, drone_time, cur_coords)
                self.drone_events.append(drone_launch_event)
                drone_time += DRONE_LAUNCH
                drone_time += self.drone_travel_times[cur_stop][self.sequence[i + 1]]
                drone_stop = self.route_data[self.route_data['stop_id'] == self.sequence[i + 1]].iloc[0]
                drone_coords = {'lat' : drone_stop['lat'], 'lng' : drone_stop['lng']}
                drone_arrive_event = Event(DRONE_DELIVERY_START, self.sequence[i + 1], drone_time, drone_coords)
                self.drone_events.append(drone_arrive_event)
                drone_time += DRONE_LAND + DRONE_LAUNCH
                self.drone_path.append(self.sequence[i + 1])
                drone_depart_event = Event(DRONE_DELIVERY_END, self.sequence[i + 1], drone_time, drone_coords)
                self.drone_events.append(drone_depart_event)
                # Drone flies to meet back up with the vehicle
                drone_time += self.drone_travel_times[self.sequence[i + 1]][self.sequence[i + 2]]
                
                # We iterate i because the vehicle will skip the stop
                i += 1
                
            else: # The driver departs
                depart_event = Event(DEPART, cur_stop, cur_time, cur_coords)
                self.vehicle_events.append(depart_event)
            
            i += 1
                
                
            
    
        
        
    
    def print_vehicle_path(self):
          print(self.vehicle_path)  # Ensure vehicle_path is correctly converted to string

    def print_vehicle_events(self):
          for event in self.vehicle_events:
              print(event)  # Convert Event object to string
    
    def print_drone_path(self):
          print(self.drone_path)  # Ensure drone_path is correctly converted to string

    def print_drone_events(self):
        for event in self.drone_events:
            print(event)
    
    def save_vehicle_path(self, path):
        os.makedirs(path, exist_ok=True)
        path_json = []
        for i, stop_id in enumerate(self.vehicle_path):
            stop = self.route_data[self.route_data['stop_id'] == stop_id].iloc[0]
            vehicle_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}
            path_json.append({"id": i+1, "stop": stop_id, "coordinates": vehicle_coords})

        with open(f"{path}/{self.route_id}_vehicle_path.json", 'w') as file:
            json.dump(path_json, file, indent=4)

    def save_drone_path(self, path):
        os.makedirs(path, exist_ok=True)
        path_json = []
        for i, stop_id in enumerate(self.drone_path):
            stop = self.route_data[self.route_data['stop_id'] == stop_id].iloc[0]
            drone_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}
            path_json.append({"id": i+1, "stop": stop_id, "coordinates": drone_coords})

        with open(f"{path}/{self.route_id}_drone_path.json", 'w') as file:
            json.dump(path_json, file, indent=4)

    def save_vehicle_events(self, path):
        os.makedirs(path, exist_ok=True)
        # Convert the list of events to a list of dictionaries and assign an ID to each
        events_with_ids = [{'id': i+1, **event.to_dict()} for i, event in enumerate(self.vehicle_events)]
        # Save the list of events as a JSON file
        with open(f"{path}/{self.route_id}_vehicle_events.json", 'w') as file:
            json.dump(events_with_ids, file, indent=4)

    def save_drone_events(self, path):
        os.makedirs(path, exist_ok=True)
        # Convert the list of events to a list of dictionaries and assign an ID to each
        events_with_ids = [{'id': i+1, **event.to_dict()} for i, event in enumerate(self.drone_events)]
        # Save the list of events as a JSON file
        with open(f"{path}/{self.route_id}_drone_events.json", 'w') as file:
            json.dump(events_with_ids, file, indent=4)


In [19]:
proposed_sequence = sequence_to_stops_for_route(route_id)
test = VRPDSolver(route_id, proposed_sequence, package_data_route_df, route_data_route_df, vehicle_travel_times_route, drone_travel_times_route)
drone_usage = test.find_drone_uses()
time_saved = test.drone_save_time(drone_usage)
best_drone_usage = test.maximize_saved_time(time_saved)
test.generate_vehicle_drone_path(best_drone_usage)

In [20]:
test.print_vehicle_events()


DEPART HI 0 {'lat': 42.031368, 'lng': -87.776596}
ARRIVE RC 281.6 {'lat': 42.034701, 'lng': -87.787183}
SERVICE_START RC 281.6 {'lat': 42.034701, 'lng': -87.787183}
SERVICE_END RC 497.1 {'lat': 42.034701, 'lng': -87.787183}
DEPART RC 497.1 {'lat': 42.034701, 'lng': -87.787183}
ARRIVE OM 513.5 {'lat': 42.0347, 'lng': -87.78652}
SERVICE_START OM 513.5 {'lat': 42.0347, 'lng': -87.78652}
SERVICE_END OM 534.0 {'lat': 42.0347, 'lng': -87.78652}
LOAD_START OM 534.0 {'lat': 42.0347, 'lng': -87.78652}
LOAD_START OM 534.0 {'lat': 42.0347, 'lng': -87.78652}
LOAD_END OM 549.0 {'lat': 42.0347, 'lng': -87.78652}
ARRIVE UB 1042.7 {'lat': 42.010624, 'lng': -87.794307}
SERVICE_START UB 1042.7 {'lat': 42.010624, 'lng': -87.794307}
SERVICE_END UB 1081.7 {'lat': 42.010624, 'lng': -87.794307}
DEPART UB 1081.7 {'lat': 42.010624, 'lng': -87.794307}
ARRIVE FM 1104.0 {'lat': 42.01097, 'lng': -87.795053}
SERVICE_START FM 1104.0 {'lat': 42.01097, 'lng': -87.795053}
SERVICE_END FM 1155.0 {'lat': 42.01097, 'lng': 

In [21]:
test.print_drone_events()

DEPART HI 0 {'lat': 42.031368, 'lng': -87.776596}
ARRIVE RC 281.6 {'lat': 42.034701, 'lng': -87.787183}
ARRIVE OM 513.5 {'lat': 42.0347, 'lng': -87.78652}
LOAD_END OM 549.0 {'lat': 42.0347, 'lng': -87.78652}
LAUNCH OM 549.0 {'lat': 42.0347, 'lng': -87.78652}
DRONE_DELIVERY_START KN 936.3 {'lat': 42.011134, 'lng': -87.793221}
DRONE_DELIVERY_END KN 946.3 {'lat': 42.011134, 'lng': -87.793221}
LAND UB 1042.7 {'lat': 42.010624, 'lng': -87.794307}
ARRIVE FM 1104.0 {'lat': 42.01097, 'lng': -87.795053}
ARRIVE LX 1227.9 {'lat': 42.009862, 'lng': -87.797879}
ARRIVE PC 1295.0 {'lat': 42.010552, 'lng': -87.798468}
ARRIVE KQ 1421.1 {'lat': 42.013021, 'lng': -87.798192}
ARRIVE RI 1559.6 {'lat': 42.01369, 'lng': -87.79821}
ARRIVE UV 1818.6 {'lat': 42.010971, 'lng': -87.801941}
ARRIVE CF 1933.8 {'lat': 42.010627, 'lng': -87.800602}
ARRIVE VG 2090.4 {'lat': 42.008005, 'lng': -87.79719}
ARRIVE OC 2249.5 {'lat': 42.007219, 'lng': -87.795648}
ARRIVE ZP 2383.8 {'lat': 42.008275, 'lng': -87.795104}
ARRIVE R

In [22]:
route_data_ordered = pd.concat([route_data_route_df[route_data_route_df['stop_id'] == stop_id] for stop_id in test.vehicle_path]).reset_index(drop=True)
len(route_data_ordered)

139

In [23]:
fig = go.Figure(go.Scattermapbox(
    lat=route_data_ordered['lat'],
    lon=route_data_ordered['lng'],
    mode='lines',
    line=dict(width=2, color='blue'),  # Customize line color and width here
    hoverinfo='none'
))
route_data_ordered['zone_id'].ffill(inplace=True)
# Add scatter points for each unique zone_id, with custom hover text
for zone_id in route_data_ordered['zone_id'].unique():
    df_sub = route_data_ordered[route_data_ordered['zone_id'] == zone_id]

    # Creating custom hover text
    hover_text = df_sub.apply(lambda row: f"Sequence: {row.name}<br> Stop ID: {row['stop_id']}<br>Zone ID: {row['zone_id']}<br>Num Packages: {row['num_packages']} <br> Service Time: {row['service_time_seconds']}", axis=1)


    fig.add_trace(go.Scattermapbox(
        lat=df_sub['lat'],
        lon=df_sub['lng'],
        mode='markers',
        marker=go.scattermapbox.Marker(size=15),
        name=str(zone_id),
        text=hover_text,
        hoverinfo='text'
    ))

# Update the layout to use OpenStreetMap style and adjust other layout properties
fig.update_layout(

    height=600,
    mapbox=dict(
        style="open-street-map",
        zoom=10,  # Adjust zoom level here
        center=dict(lat=route_data_ordered['lat'].mean(), lon=route_data_ordered['lng'].mean())
    ),
    showlegend=False  # Set to True if you want to show legend
)
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))


# Show the figure
fig.show()

In [24]:
drone_data_ordered = pd.concat([route_data_route_df[route_data_route_df['stop_id'] == stop_id] for stop_id in test.drone_path]).reset_index(drop=True)
len(drone_data_ordered)

164

In [25]:
fig = go.Figure(go.Scattermapbox(
    lat=drone_data_ordered['lat'],
    lon=drone_data_ordered['lng'],
    mode='lines',
    line=dict(width=2, color='blue'),  # Customize line color and width here
    hoverinfo='none'
))
drone_data_ordered['zone_id'].ffill(inplace=True)
# Add scatter points for each unique zone_id, with custom hover text
for zone_id in drone_data_ordered['zone_id'].unique():
    df_sub = drone_data_ordered[drone_data_ordered['zone_id'] == zone_id]

    # Creating custom hover text
    hover_text = df_sub.apply(lambda row: f"Sequence: {row.name}<br> Stop ID: {row['stop_id']}<br>Zone ID: {row['zone_id']}<br>Num Packages: {row['num_packages']} <br> Service Time: {row['service_time_seconds']}", axis=1)


    fig.add_trace(go.Scattermapbox(
        lat=df_sub['lat'],
        lon=df_sub['lng'],
        mode='markers',
        marker=go.scattermapbox.Marker(size=15),
        name=str(zone_id),
        text=hover_text,
        hoverinfo='text'
    ))

# Update the layout to use OpenStreetMap style and adjust other layout properties
fig.update_layout(

    height=600,
    mapbox=dict(
        style="open-street-map",
        zoom=10,  # Adjust zoom level here
        center=dict(lat=drone_data_ordered['lat'].mean(), lon=drone_data_ordered['lng'].mean())
    ),
    showlegend=False  # Set to True if you want to show legend
)
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))


# Show the figure
fig.show()

In [26]:
# Create a figure
fig = go.Figure()

# Add the drone path
fig.add_trace(go.Scattermapbox(
    lat=drone_data_ordered['lat'],
    lon=drone_data_ordered['lng'],
    mode='lines',
    line=dict(width=2, color='red'),
    hoverinfo='none',
    name='Drone Path'
))

# Add scatter points for each unique zone_id in the drone data
for zone_id in drone_data_ordered['zone_id'].unique():
    df_sub = drone_data_ordered[drone_data_ordered['zone_id'] == zone_id]
    hover_text = df_sub.apply(lambda row: f"Sequence: {row.name}<br> Stop ID: {row['stop_id']}<br>Zone ID: {row['zone_id']}<br>Num Packages: {row['num_packages']} <br> Service Time: {row['service_time_seconds']}", axis=1)
    fig.add_trace(go.Scattermapbox(
        lat=df_sub['lat'],
        lon=df_sub['lng'],
        mode='markers',
        marker=go.scattermapbox.Marker(size=15),
        name=f"Drone Zone {zone_id}",
        text=hover_text,
        hoverinfo='text'
    ))


# Add the route path
fig.add_trace(go.Scattermapbox(
    lat=route_data_ordered['lat'],
    lon=route_data_ordered['lng'],
    mode='lines',
    line=dict(width=2, color='blue'),
    hoverinfo='none',
    name='Route Path'
))

# Add scatter points for each unique zone_id in the route data
for zone_id in route_data_ordered['zone_id'].unique():
    df_sub = route_data_ordered[route_data_ordered['zone_id'] == zone_id]
    hover_text = df_sub.apply(lambda row: f"Sequence: {row.name}<br> Stop ID: {row['stop_id']}<br>Zone ID: {row['zone_id']}<br>Num Packages: {row['num_packages']} <br> Service Time: {row['service_time_seconds']}", axis=1)
    fig.add_trace(go.Scattermapbox(
        lat=df_sub['lat'],
        lon=df_sub['lng'],
        mode='markers',
        marker=go.scattermapbox.Marker(size=15),
        name=f"Route Zone {zone_id}",
        text=hover_text,
        hoverinfo='text'
    ))



# Update the layout to use OpenStreetMap style and adjust other layout properties
fig.update_layout(
    height=600,
    mapbox=dict(
        style="open-street-map",
        zoom=10,
        center=dict(
            lat=(route_data_ordered['lat'].mean() + drone_data_ordered['lat'].mean()) / 2,
            lon=(route_data_ordered['lng'].mean() + drone_data_ordered['lng'].mean()) / 2
        )
    ),
    showlegend=True
)

fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))

# Show the figure
fig.show()

In [27]:
proposed_sequences = json.load(open('proposed_sequences.json'))
proposed_route_ids = list(proposed_sequences.keys())

for route_id in proposed_route_ids:
    package_data_route_df, route_data_route_df, vehicle_travel_times_route = get_data_for_route(route_id)
    route_data_route_df['zone_id'].ffill(inplace=True)
    drone_travel_times_route = calculate_drone_travel_time(route_data_route_df)
    package_data_route_df['drone_possible'] = (package_data_route_df['depth_cm'] < DRONE_DEPTH) & (package_data_route_df['height_cm'] < DRONE_HEIGHT) & (package_data_route_df['width_cm'] < DRONE_WIDTH)
    
    package_info_agg = package_data_route_df.groupby('stop_id').apply(
    lambda x: pd.Series({
        'packages': x[['package_id', 'depth_cm', 'height_cm', 'width_cm']].to_dict('records'),
        'num_packages': len(x),
        'drone_possible': (len(x) == 1) and (x['drone_possible'].iloc[0] == True),
        'service_time_seconds': x['planned_service_time_seconds'].mean() # Not sure if this should be sum or average. https://github.com/MIT-CAVE/rc-cli/blob/main/templates/data_structures.md
    })
    ).reset_index()
    
    route_data_route_df = pd.merge(route_data_route_df, package_info_agg, how='left', on='stop_id')
    
    proposed_sequence = sequence_to_stops_for_route(route_id)
    
    solver = VRPDSolver(route_id, proposed_sequence, package_data_route_df, route_data_route_df, vehicle_travel_times_route, drone_travel_times_route)
    drone_usage = solver.find_drone_uses()
    time_saved = solver.drone_save_time(drone_usage)
    best_drone_usage = solver.maximize_saved_time(time_saved)
    solver.generate_vehicle_drone_path(best_drone_usage)
    solver.save_vehicle_path('optimized_vehicle_paths')
    solver.save_drone_path('optimized_drone_paths')
    solver.save_vehicle_events('optimized_vehicle_events')
    solver.save_drone_events('optimized_drone_events')
    